In [1]:
%cd ../

%load_ext autoreload
%autoreload 2

/Users/hoangle/Uni/Thesis


/Users/hoangle/Uni/Thesis/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
from collections import defaultdict

import pandas as pd
import numpy as np
import yaml

from recbole.config import Config
from recbole.data import (
    create_dataset,
    data_preparation,
)
from tqdm.notebook import tqdm
from recbole.utils import init_seed, ModelType

from src.utils import Paths
from src.real_temporal import SimulatedOnlineDataset, SimulatedOnlineSequentialDataset

# Get suitable cutoff

In [4]:
def get_suitable_cutoff(ds_name: str, model_name: str = "S3Rec", scheme: str = "so") -> tuple:
    """Get suitable cutoff timestamp: at which there are the most active users

    Args:
        ds_name (str): dataset name

    Returns:
        tuple: suitable timestamp and the number of active users
    """

    # Get dataset without normalizing the timestamp
    config_dict = {
        'normalize_all': False,
        'user_inter_num_interval': "[5,inf)",
        'item_inter_num_interval': "[5,inf)",
        'device': 'cpu',
        'use_gpu': False,

        'train_neg_sample_args': None,

        'eval_args': {
            "order": "TO",
            "split": { "LS": "valid_and_test" },
            "group_by": None,
            'mode': 'pop100'
        },
    }

    # Get additional dataset-specific config
    paths = Paths(model_name, ds_name, scheme)

    with open(paths.get_path_dataset_conf()) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
    config_dict.update(data)

    config = Config(
        model=model_name,
        dataset=ds_name,
        config_dict=config_dict,
    )
    init_seed(config["seed"], config["reproducibility"])
    df = create_dataset(config).inter_feat.copy()

    # Create dataframe of users and corresponding first/last timestamp
    user_max_ts = df.groupby('user_id')['timestamp'].max()
    user_min_ts = df.groupby('user_id')['timestamp'].min()
    df_user = pd.DataFrame(
        {
            'max': user_max_ts,
            'min': user_min_ts,
        },
        index=user_max_ts.index
    )

    counts = defaultdict(int) 
    for ts in df_user['min']:
        counts[ts] += 1
    for ts in df_user['max']:
        counts[ts] -= 1

    timestamps = sorted(counts.keys())
    accum = {}

    s = 0
    for ts in timestamps:
        s += counts[ts]
        accum[ts] = s
    series = pd.Series(accum)

    suitable_ts = series.idxmax()
    max_active_user = series[suitable_ts]

    return suitable_ts, max_active_user

# ds = "ml-100k"
# ds = "amazon-digital-music"

# ds = "ml-1m"
ds = "ml-20m"
# ds = "yelp"
# ds = "steam"
print(f"{ds}: {get_suitable_cutoff(ds)}")

ParserError: '	' expected after '"'